## Qu'est ce que le clustering ? ##
=======================================

En data-science, le clustering est une technique de regroupement d'un ensemble de données en sous-ensembles ("clusters") de façon à ce que les données d'un même sous-ensemble soient plus similaires entre elles qu'avec celles des autres autres sous-ensembles. C'est une technique non-supervisée car elle ne nécessite pas de données de d'entraienement avec des résultats connus.



## Est ce un problème difficile ? pourquoi ? Donnez la complexité en temps et mémoire ##
=======================================

Oui, de mon point de vue, c'est un problème difficile :
- déterminer le nombre de clusters peut être compliqué lorsque l'on ne le connait pas à l'avance. On utilise la méthode du coude ou celle de la silhouette pour le déterminer au mieux mais cela peut être couteux en temps et en calculs.
- Optimiser le clustering (faire le tri au mieux) est très complexe => concept de problème NP-difficile
- la dimensionalité des données : plus il y a de dimensions, plus les calculs sont importants : mémoire et temps



## Quelle sont les métriques utilisés pour le clustering ? ##
=======================================

Coefficient de Silhouette, Somme des carrés intra-cluster, somme des carrés entre les clusters, indice de Davies-Bouldin, ARI, Pureté, F1-score



## 3 métriques avec ground truth (dont MNI) et 3 sans ground truth (dont silhouette) ##
=======================================

### Avec ground truth (avec connaissance terrain) ###
- ARI, Pureté, NMI (entre 0 et 1), AMI
### Sans ground truth (avec connaissance terrain) ###
- silhouette, WCSS, indice de Davies-Bouldin

NMI et AMI sont des métriques avec ground truth (comparaison des vrais clusters aux clusters prédits). Le coéficient silhouette évalue la qualité du clustering uniquement à partir des données, sans information extérieure.


=======================================


# 20_newgroups #

## K-means & TfidfVectorizer ##

In [16]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from sklearn.metrics import normalized_mutual_info_score, adjusted_mutual_info_score, silhouette_score


newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes')) # pour ne récupérer qu'une partie du dataset (pour me faciliter la vie et les temps de calculs)

vectorizer = TfidfVectorizer(stop_words='english') # instanciation  de l'algo de vectorisation, en supprimant les mots inutiles
vectors = vectorizer.fit_transform(newsgroups.data) # vectorisation des données

X_train, X_test, y_train, y_test = train_test_split(vectors, newsgroups.target, test_size=0.3, random_state=42)

num_clusters = 20 # il y a 20 newsgroups
km = KMeans(n_clusters=num_clusters, n_init=10, random_state=42) # clsutering
km.fit(X_train)

predicted_clusters = km.predict(X_test)

# Calculez les métriques
nmi = normalized_mutual_info_score(y_test, predicted_clusters)
ami = adjusted_mutual_info_score(y_test, predicted_clusters)
silhouette = silhouette_score(X_test, predicted_clusters)

# Affichez les résultats
print(f"NMI: {nmi:.2f}")
print(f"AMI: {ami:.2f}")
print(f"Silhouette Score: {silhouette:.2f}")

NMI: 0.30
AMI: 0.29
Silhouette Score: 0.00


## K-means & CountVectorizer ##

In [17]:
# 2ème façon
vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups.data)


X_train, X_test, y_train, y_test = train_test_split(vectors, newsgroups.target, test_size=0.3, random_state=42)


num_clusters = 20


km = KMeans(n_clusters=num_clusters, n_init=10, random_state=42)
km.fit(X_train)


predicted_clusters = km.predict(X_test)


nmi = normalized_mutual_info_score(y_test, predicted_clusters)
ami = adjusted_mutual_info_score(y_test, predicted_clusters)
silhouette = silhouette_score(X_test, predicted_clusters)

print(f"NMI: {nmi:.2f}")
print(f"AMI: {ami:.2f}")
print(f"Silhouette Score: {silhouette:.2f}")


NMI: 0.00
AMI: 0.00
Silhouette Score: 0.91


## Constatations ##

Les silhouette Score sont complètement opposés. NMI et AMI diffèrent d'une vectorisation à une autre mais ne sont jamais concluants.

## LDA & TfidfVectorizer ##

In [18]:
vectorizer = TfidfVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups.data)


X_train, X_test, y_train, y_test = train_test_split(vectors, newsgroups.target, test_size=0.3, random_state=42)


num_topics = 20


lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X_train)


document_topics = lda.transform(X_test)
predicted_clusters = document_topics.argmax(axis=1)


nmi = normalized_mutual_info_score(y_test, predicted_clusters)
ami = adjusted_mutual_info_score(y_test, predicted_clusters)
silhouette = silhouette_score(X_test, predicted_clusters)

print(f"NMI: {nmi:.2f}")
print(f"AMI: {ami:.2f}")
print(f"Silhouette Score: {silhouette:.2f}")

NMI: 0.04
AMI: 0.02
Silhouette Score: -0.21


## LDA & CountVectorizer ##

In [19]:
from sklearn.metrics import classification_report

vectorizer = CountVectorizer(stop_words='english')
vectors = vectorizer.fit_transform(newsgroups.data)


X_train, X_test, y_train, y_test = train_test_split(vectors, newsgroups.target, test_size=0.3, random_state=42)


num_topics = 20


lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X_train)


document_topics = lda.transform(X_test)
predicted_clusters = document_topics.argmax(axis=1)


nmi = normalized_mutual_info_score(y_test, predicted_clusters)
ami = adjusted_mutual_info_score(y_test, predicted_clusters)
silhouette = silhouette_score(X_test, predicted_clusters)

print(f"NMI: {nmi:.2f}")
print(f"AMI: {ami:.2f}")
print(f"Silhouette Score: {silhouette:.2f}")

report = classification_report(y_test, predicted_clusters, target_names=newsgroups.target_names, zero_division=0)
print(report)


NMI: 0.32
AMI: 0.31
Silhouette Score: -0.22
                          precision    recall  f1-score   support

             alt.atheism       0.01      0.04      0.02       135
           comp.graphics       0.20      0.01      0.01       166
 comp.os.ms-windows.misc       0.00      0.00      0.00       170
comp.sys.ibm.pc.hardware       0.04      0.01      0.01       182
   comp.sys.mac.hardware       0.00      0.00      0.00       183
          comp.windows.x       0.01      0.01      0.01       169
            misc.forsale       0.00      0.00      0.00       172
               rec.autos       0.08      0.01      0.02       191
         rec.motorcycles       0.42      0.22      0.29       198
      rec.sport.baseball       0.02      0.01      0.02       168
        rec.sport.hockey       0.00      0.02      0.01       163
               sci.crypt       0.00      0.00      0.00       195
         sci.electronics       0.08      0.01      0.02       177
                 sci.med       

## Constatations ##

Aucun score n'est concluant. Je suis paumé ;-|